# QONNX - Analysis Passes
--------------------------------------
This notebook is about analysis passes in QONNX. The procedure for creating an analysis pass is shown using an example.

We'll use the following utility functions to print the source code for function calls (`showSrc()`) and to visualize a network using netron (`showInNetron()`) in the Jupyter notebook:

In [ ]:
import inspect
import netron
import os
from IPython.display import IFrame
import urllib.request


def showSrc(what):
    print("".join(inspect.getsourcelines(what)[0]))


def showInNetron(model_filename: str, localhost_url: str = None, port: int = None):
    """Shows a ONNX model file in the Jupyter Notebook using Netron.

    :param model_filename: The path to the ONNX model file.
    :type model_filename: str

    :param localhost_url: The IP address used by the Jupyter IFrame to show the model.
     Defaults to localhost.
    :type localhost_url: str, optional

    :param port: The port number used by Netron and the Jupyter IFrame to show
     the ONNX model.  Defaults to 8081.
    :type port: int, optional

    :return: The IFrame displaying the ONNX model.
    :rtype: IPython.lib.display.IFrame
    """
    try:
        port = port or int(os.getenv("NETRON_PORT", default="8081"))
    except ValueError:
        port = 8081
    localhost_url = localhost_url or os.getenv("LOCALHOST_URL", default="localhost")
    netron.start(model_filename, address=("0.0.0.0", port), browse=False)
    return IFrame(src=f"http://{localhost_url}:{port}/", width="100%", height=400)

def download_model_from_zoo():
    qonnx_url="https://github.com/fastmachinelearning/QONNX_model_zoo/raw/main/models/MNIST/Brevitas_FINN_TFC/TFC/TFC_2W2A.onnx"
    dl_file="TFC_2W2A.onnx"
    urllib.request.urlretrieve(qonnx_url, dl_file)

## General Information
------------------------------
* traverses the graph structure and produces information about certain properties
* input: ModelWrapper
* returns dictionary of named properties that the analysis extracts

### Example - Quantity analysis of operation types
As an example, an analysis is designed that returns the number of nodes of the same operation types.

First the model is shown to illustrate the analysis. For this Netron is used. Netron is a visualizer for neural network, deep learning and machine learning models.

In [ ]:
download_model_from_zoo()
showInNetron("TFC_2W2A.onnx")

The model has to be loaded to process it with QONNX utils. This is done with `ModelWrapper`. As described in the short introduction, this is the format an analysis pass takes as input.

In [ ]:
from qonnx.core.modelwrapper import ModelWrapper
model = ModelWrapper('TFC_2W2A.onnx')

The idea is to count all nodes that have the same operation type. The result should contain the operation types and the corresponding number of nodes that occur in the model. In the beginning an empty dictionary is created which is filled by the function and returned as result to the user at the end of the analysis.

In [ ]:
def count_unique_node_types(model):
    count_dict = {}
    for node in model.graph.node:
        if node.op_type in count_dict:
            count_dict[node.op_type] +=1
        else:
            count_dict[node.op_type] = 1
    return count_dict

The function takes the model as input and iterates over the nodes. Then it is checked whether there is already an entry for the operation type in the dictionary. If this is not the case, an entry is created and set to `1`. If there is already an entry, it is incremented. If all nodes in the model have been iterated, the filled dictionary is returned.

The analysis function of ModelWrapper is used to perform the analysis just designed. It is shown below and takes the function as input and performs it by passing the model to the function.

In [ ]:
showSrc(ModelWrapper.analysis)

The result can now simply be determined by calling the `.analysis` function.

In [ ]:
print(model.analysis(count_unique_node_types))